In [1]:
!pip install transformers datasets peft accelerate bitsandbytes evaluate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset


dataset = load_dataset(
    "json",
    data_files="/content/qa_dataset (2).json"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# Load model and tokenizer
model_id = "deepseek-ai/deepseek-coder-1.3b-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")

# ✅ Add pad token fix
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Your dataset
# dataset = load_dataset("bitext/Bitext-retail-ecommerce-llm-chatbot-training-dataset", split="train[:500]")

# Preprocess function
def preprocess_function(example):
    question = example.get("instruction") or example.get("input") or example.get("question")
    answer = example.get("response") or example.get("output") or example.get("answer")

    text = f"### Question:\n{question}\n\n### Answer:\n{answer}"
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=False)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05
)
model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=200,
    logging_dir="./logs",
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(800)),
    eval_dataset=tokenized_dataset["train"].select(range(800, 1000))
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=400, training_loss=25.0563623046875, metrics={'train_runtime': 636.2177, 'train_samples_per_second': 1.257, 'train_steps_per_second': 0.629, 'total_flos': 3154470607257600.0, 'train_loss': 25.0563623046875, 'epoch': 1.0})

In [7]:
!pip install --upgrade transformers

In [10]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./deepseek-finetuned-lora")
tokenizer.save_pretrained("./deepseek-finetuned-lora")

('./deepseek-finetuned-lora/tokenizer_config.json',
 './deepseek-finetuned-lora/special_tokens_map.json',
 './deepseek-finetuned-lora/tokenizer.json')

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

base_model_id = "deepseek-ai/deepseek-coder-1.3b-base"
fine_tuned_dir = "./deepseek-finetuned-lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_dir)

# Load base model
model = AutoModelForCausalLM.from_pretrained(base_model_id, load_in_4bit=True, device_map="auto")

# Load LoRA fine-tuned weights on top of base model
model = PeftModel.from_pretrained(model, fine_tuned_dir)

# Add pad token fix again
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModelForCausalLM
import torch

base_model_id = "deepseek-ai/deepseek-coder-1.3b-base"
fine_tuned_dir = "./deepseek-finetuned-lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_dir)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    load_in_4bit=True,
    device_map="auto"
)

# ✅ Load LoRA fine-tuned weights correctly
model = PeftModelForCausalLM.from_pretrained(model, fine_tuned_dir)

# Pad token fix
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Use pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Try a question
prompt = "### Question:\nDoes Sony laptops have warranty?\n\n### Answer:\n"

output = pipe(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print(output[0]["generated_text"])


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


### Question:
Does Sony laptops have warranty?

### Answer:
All Sony laptops come with manufacturer warranty for 1 year.



In [15]:
!pip install bert_score
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e75985b110fb170136eccd4ea954b8b8c062417412d18b3f9cb7f0bf5df824af
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 1000
    })
})


In [17]:
print(dataset["train"][:5])

{'instruction': ['What is the warranty period for Sony laptops?', 'Do you sell Huawei headphones?', 'What is your return policy?', 'What is the warranty period for Samsung fridges?', 'What is the warranty period for Xiaomi microwaves?'], 'response': ['All Sony laptops come with a 3 years manufacturer warranty covering factory defects.', 'Yes, ClickBuy offers Huawei headphones with full specifications and competitive pricing.', 'You can return any product within 14 days of purchase if it’s unused and in original packaging.', 'All Samsung fridges come with a 1 year manufacturer warranty covering factory defects.', 'All Xiaomi microwaves come with a 1 year manufacturer warranty covering factory defects.']}


In [18]:
from evaluate import load
import pandas as pd
import os

# Load evaluation metrics
bleu = load("bleu")
rouge = load("rouge")
bertscore = load("bertscore")

# Example: Replace with your actual model predictions and true answers
preds = [
    "Does Sony laptops have warranty?",
    "All Sony laptops come with manufacturer warranty for 1 year."
]
refs = [
    ["What is the warranty period for Sony laptops?"],
    ["All Sony laptops come with a 3 years manufacturer warranty covering factory defects."]
]

# Compute metrics
bleu_result = bleu.compute(predictions=preds, references=refs)
# Modify ROUGE computation to handle different return types
rouge_result = rouge.compute(predictions=preds, references=[r[0] for r in refs])
rouge_l_fmeasure = rouge_result["rougeL"].mid.fmeasure if hasattr(rouge_result["rougeL"], 'mid') else rouge_result["rougeL"]

bertscore_result = bertscore.compute(predictions=preds, references=[r[0] for r in refs], lang="en")

# Combine results neatly
results = {
    "BLEU": [bleu_result["bleu"]],
    "ROUGE-L": [rouge_l_fmeasure],
    "BERTScore (F1)": [sum(bertscore_result["f1"]) / len(bertscore_result["f1"])],
}

# Convert to DataFrame
df = pd.DataFrame(results)

# Save results
os.makedirs("results", exist_ok=True)
df.to_csv("results/metrics.csv", index=False)

print(df)
print("✅ Metrics saved to results/metrics.csv")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


       BLEU   ROUGE-L  BERTScore (F1)
0  0.231804  0.458194        0.944197
✅ Metrics saved to results/metrics.csv


In [19]:
import math
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "./deepseek-finetuned-lora"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text = "### Question:\nWhat is the return policy?\n\n### Answer:\nYou can return items within 14 days."
inputs = tokenizer(text, return_tensors="pt")
loss = model(**inputs, labels=inputs["input_ids"]).loss
perplexity = math.exp(loss.item())
print("Perplexity:", perplexity)

Perplexity: 2.6796218734130317
